In [ ]:
!pip install langchain langchain-core langchain-community
!pip install langchain-text-splitters
!pip install faiss-cpu
!pip install unstructured pypdf
!pip install python-dotenv requests

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
import requests
requests.get("http://localhost:11434")

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="qwen2:0.5b",  # change to mistral, phi3, etc.
    temperature=0.2,
)

In [ ]:
response = llm.invoke("Explain machine learning in simple words.")
print(response)

In [ ]:
#Load Speech.txt or any txt file
loader = TextLoader("Speech.txt")
docs = loader.load()

In [ ]:
#splits
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [ ]:
# Embeddings
emb = OllamaEmbeddings(model="qwen2:0.5b")

# Vector DB
db = FAISS.from_documents(chunks, emb)
retriever = db.as_retriever()

In [ ]:
#llm
llm = Ollama(model="qwen2:0.5b")

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question:

Context:
{context}

Question:
{question}

Answer briefly:
""")

# Build LCEL RAG Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
# Query
response = rag_chain.invoke("Summarize the document in 5 lines.")

print("\n📌 RAG Output:\n", response)